In [1]:
import csv
from datetime import datetime
import os

EXPENSES_FILE = 'expenses.csv'

def load_expenses():
    """Loads expenses from the CSV file."""
    expenses = []
    if os.path.exists(EXPENSES_FILE):
        with open(EXPENSES_FILE, mode='r', newline='') as file:
            reader = csv.DictReader(file)
            for row in reader:
                try:
                    # Ensure 'Amount' is converted to float
                    row['Amount'] = float(row['Amount'])
                    expenses.append(row)
                except ValueError:
                    print(f"Skipping row with invalid amount: {row}")
    return expenses

def save_expenses(expenses):
    """Saves expenses to the CSV file."""
    # Define fieldnames to ensure consistent order in CSV
    fieldnames = ['Date', 'Category', 'Amount', 'Description']

    with open(EXPENSES_FILE, mode='w', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(expenses)

def get_monthly_budget():
    """Prompts the user to enter their monthly budget."""
    while True:
        try:
            budget = float(input("Enter your monthly budget (₹): "))
            if budget < 0:
                print("Budget cannot be negative. Please enter a positive value.")
            else:
                return budget
        except ValueError:
            print("Invalid input. Please enter a numerical value for the budget.")

def add_new_expense(expenses):
    """Allows the user to add a new expense."""
    print("\n--- Add New Expense ---")
    date_str = input("Enter date (YYYY-MM-DD, default: today): ")
    if not date_str:
        date_str = datetime.today().strftime('%Y-%m-%d')
    else:
        try:
            datetime.strptime(date_str, '%Y-%m-%d')
        except ValueError:
            print("Invalid date format. Using today's date.")
            date_str = datetime.today().strftime('%Y-%m-%d')

    category = input("Enter category (e.g., Food, Travel, Bills): ").strip()
    if not category:
        print("Category cannot be empty. Please enter a category.")
        return # Exit the function if category is empty

    while True:
        try:
            amount = float(input("Enter amount: "))
            if amount <= 0:
                print("Amount must be positive. Please enter a valid amount.")
            else:
                break
        except ValueError:
            print("Invalid input. Please enter a numerical value for the amount.")

    description = input("Enter description (optional): ").strip()

    expense = {
        'Date': date_str,
        'Category': category,
        'Amount': amount,
        'Description': description
    }
    expenses.append(expense)
    print("Expense added successfully!")

def view_all_expenses(expenses):
    """Displays all stored expenses in a tabular format."""
    print("\n--- All Expenses ---")
    if not expenses:
        print("No expenses recorded yet.")
        return

    # Print header
    print(f"{'Date':<12} {'Category':<15} {'Amount':<10} {'Description':<30}")
    print("-" * 67) # Separator line

    # Print each expense
    for expense in expenses:
        print(f"{expense['Date']:<12} {expense['Category']:<15} {expense['Amount']:<10.2f} {expense['Description']:<30}")

def view_expenses_by_category(expenses):
    """Allows the user to filter and view expenses by category."""
    print("\n--- View Expenses by Category ---")
    if not expenses:
        print("No expenses recorded yet.")
        return

    category_filter = input("Enter category name to filter by: ").strip()
    # Filter expenses (case-insensitive)
    filtered_expenses = [e for e in expenses if e['Category'].lower() == category_filter.lower()]

    if not filtered_expenses:
        print(f"No expenses found for category '{category_filter}'.")
        return

    print(f"\nExpenses for category '{category_filter}':")
    # Print header for filtered expenses
    print(f"{'Date':<12} {'Category':<15} {'Amount':<10} {'Description':<30}")
    print("-" * 67)
    for expense in filtered_expenses:
        print(f"{expense['Date']:<12} {expense['Category']:<15} {expense['Amount']:<10.2f} {expense['Description']:<30}")

def view_monthly_summary(expenses, monthly_budget):
    """Shows total expenses for a selected month and compares with budget."""
    print("\n--- Monthly Summary ---")
    if not expenses:
        print("No expenses recorded yet.")
        return

    while True:
        month_input = input("Enter month (MM) for summary: ").strip()
        year_input = input("Enter year (YYYY) for summary: ").strip()
        if not month_input.isdigit() or not year_input.isdigit() or not (1 <= int(month_input) <= 12):
            print("Invalid month or year. Please enter numerical values (MM) for month and (YYYY) for year.")
        else:
            break

    total_spent_month = 0.0
    for expense in expenses:
        try:
            exp_date = datetime.strptime(expense['Date'], '%Y-%m-%d')
            if exp_date.month == int(month_input) and exp_date.year == int(year_input):
                total_spent_month += expense['Amount']
        except ValueError:
            # Handle cases where date in CSV might be malformed
            print(f"Warning: Skipping expense with malformed date: {expense['Date']}")


    print(f"\nSummary for {month_input}/{year_input}:")
    print(f"Total spent: ₹{total_spent_month:.2f}")
    print(f"Monthly Budget: ₹{monthly_budget:.2f}")

    if total_spent_month > monthly_budget:
        print(f"⚠️ Warning: You have exceeded your monthly budget by ₹{(total_spent_month - monthly_budget):.2f}!")
    elif total_spent_month < monthly_budget:
        print(f"You are ₹{(monthly_budget - total_spent_month):.2f} under your monthly budget.")
    else:
        print("You have spent exactly your monthly budget.")

def show_full_expense_summary(expenses):
    """Shows total spend in each category and overall total."""
    print("\n--- Full Expense Summary by Category ---")
    if not expenses:
        print("No expenses recorded yet.")
        return

    category_totals = {}
    overall_total = 0.0

    for expense in expenses:
        category = expense['Category']
        amount = expense['Amount']
        category_totals[category] = category_totals.get(category, 0.0) + amount
        overall_total += amount

    print("\nTotal spend by Category:")
    for category, total in category_totals.items():
        print(f"- {category}: ₹{total:.2f}")

    print(f"\nOverall Total Expenses: ₹{overall_total:.2f}")

def main():
    """Main function to run the Personal Expense Tracker."""
    print("🧾 Welcome to Personal Expense Tracker 🧾")

    expenses = load_expenses()
    monthly_budget = get_monthly_budget()

    while True:
        print("\n=== Expense Tracker Menu ===")
        print("1. Add New Expense")
        print("2. View All Expenses")
        print("3. View Expenses by Category")
        print("4. View Monthly Summary")
        print("5. Show Full Expense Summary")
        print("6. Save & Exit")

        choice = input("Enter your choice (1-6): ")

        if choice == '1':
            add_new_expense(expenses)
        elif choice == '2':
            view_all_expenses(expenses)
        elif choice == '3':
            view_expenses_by_category(expenses)
        elif choice == '4':
            view_monthly_summary(expenses, monthly_budget)
        elif choice == '5':
            show_full_expense_summary(expenses)
        elif choice == '6':
            save_expenses(expenses)
            print("Exiting Expense Tracker. Your data has been saved.")
            break
        else:
            print("Invalid choice. Please enter a number between 1 and 6.")

if __name__ == "__main__":
    main()

🧾 Welcome to Personal Expense Tracker 🧾


Enter your monthly budget (₹):  25000



=== Expense Tracker Menu ===
1. Add New Expense
2. View All Expenses
3. View Expenses by Category
4. View Monthly Summary
5. Show Full Expense Summary
6. Save & Exit


Enter your choice (1-6):  2



--- All Expenses ---
Date         Category        Amount     Description                   
-------------------------------------------------------------------
2025-05-21   travel          800.00                                   
2025-05-21   food            21000.00                                 

=== Expense Tracker Menu ===
1. Add New Expense
2. View All Expenses
3. View Expenses by Category
4. View Monthly Summary
5. Show Full Expense Summary
6. Save & Exit


Enter your choice (1-6):  5



--- Full Expense Summary by Category ---

Total spend by Category:
- travel: ₹800.00
- food: ₹21000.00

Overall Total Expenses: ₹21800.00

=== Expense Tracker Menu ===
1. Add New Expense
2. View All Expenses
3. View Expenses by Category
4. View Monthly Summary
5. Show Full Expense Summary
6. Save & Exit


Enter your choice (1-6):  6


Exiting Expense Tracker. Your data has been saved.
